# Notebook for exploring how to change the backbone and head of YOLOv8

#### First install requirements

In [ ]:
!pip install -r ../requirements.txt

In [8]:
from ultralytics import YOLO
import torch
import torch.nn as nn
from torchvision.models import resnet50

# Load the pretrained YOLOv8 model
model = YOLO('yolov8n.pt')

# Define your custom backbone (e.g., ResNet-50)
class CustomBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        # Load pretrained ResNet50 backbone
        backbone = resnet50(pretrained=True)
        self.conv1 = backbone.conv1
        self.bn1 = backbone.bn1
        self.relu = backbone.relu
        self.maxpool = backbone.maxpool
        # Extract layers from ResNet
        self.layer1 = backbone.layer1
        self.layer2 = backbone.layer2
        self.layer3 = backbone.layer3
        self.layer4 = backbone.layer4

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # Outputs from different layers
        x1 = self.layer1(x)  # small-scale features
        x2 = self.layer2(x1) # medium-scale features
        x3 = self.layer3(x2) # large-scale features
        x4 = self.layer4(x3) # extra-large-scale features

        # YOLOv8 usually expects multiple-scale features.
        # Choose appropriate layers matching YOLOv8 neck:
        return [x2, x3, x4]  # Adjust layers to match expected YOLO neck inputs

# Replace YOLO backbone with custom backbone
model.model.model.backbone = CustomBackbone()

# Check your new architecture
print(model.model)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    

In [9]:
#change head of yolo model
import torch
import torch.nn as nn
from ultralytics.nn.modules import Conv, Detect
from ultralytics.nn.tasks import DetectionModel

class CustomDetect(nn.Module):
    def __init__(self, nc=80, ch=()):
        super().__init__()
        self.nc = nc
        self.no = nc + 4
        self.cv2 = nn.ModuleList(
            nn.Sequential(
                Conv(x, x, 3),
                Conv(x, self.no, 1)
            ) for x in ch
        )

    def forward(self, x):
        return [cv(xi) for cv, xi in zip(self.cv2, x)]

model = YOLO('yolov8n.pt')
model.model.model.head = CustomDetect(nc=80, ch=[128, 256, 512])


In [10]:
import comet_ml
from ultralytics import YOLO
COMET_API_KEY= "XLpiDIPRdxY4Pk1ObpR35JhQu"#this is an api code pls don't commit publicly
comet_ml.login(project_name="comet_yolo_changehead")

#model = YOLO("yolo11n.pt")
myPath = "/content/drive/MyDrive/5 cats.v2i.yolov11 (1)/data.yaml"
results = model.train(data=myPath, epochs=5, imgsz=640, project="comet_yolo_change_head",
    batch=32,
    save_period=1,
    save_json=True,

)

Ultralytics 8.3.95 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/5 cats.v2i.yolov11 (1)/data.yaml, epochs=5, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=1, cache=False, device=None, workers=8, project=comet_yolo_v11, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=True, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/dtaghva2001/comet-yolo-changehead/f28c1aafab8948bfba3c6915ab07dace



TensorBoard: Start with 'tensorboard --logdir comet_yolo_v11/train3', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /content/drive/MyDrive/5 cats.v2i.yolov11 (1)/train/labels.cache... 9 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all          1          2    0.00667          1     0.0742     0.0233



5 epochs completed in 0.016 hours.
Optimizer stripped from comet_yolo_v11/train3/weights/last.pt, 6.2MB
Optimizer stripped from comet_yolo_v11/train3/weights/best.pt, 6.2MB

Validating comet_yolo_v11/train3/weights/best.pt...
Ultralytics 8.3.95 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.32it/s]


                   all          1          2    0.00667          1     0.0742     0.0233
Speed: 2.1ms preprocess, 274.1ms inference, 0.0ms loss, 4.6ms postprocess per image
Saving comet_yolo_v11/train3/predictions.json...
Results saved to comet_yolo_v11/train3


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : territorial_vole_241
COMET INFO:     url                   : https://www.comet.com/dtaghva2001/comet-yolo-changehead/f28c1aafab8948bfba3c6915ab07dace
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [11]               : (0.0, 2.4360000000000004e-05)
COMET INFO:     lr/pg1 [11]               : (0.0, 2.4360000000000004e-05)
COMET INFO:     lr/pg2 [11]               : (0.0, 2.4360000000000004e-05)
COMET INFO:     metrics/mAP50(B) [12]     : (0.05585, 0.07422305764411029)
COMET INFO:     metrics/mAP50-95(B) [12]  : (0.01756, 0.02332)
COMET INFO:     metrics/precision(B) [12] : (0.00333, 0.00667)
COMET INFO:     metrics/recall(B) [12]    :